# BMEB W4020: Circuits in the Brain 2020 Fall Homework \#2
*Author:* Tingkai Liu <tl2747@columbia.edu>

*Based on Previous Work by:* Mehmet Kerem <mkt2126@columbia.edu>, Chung-Heng Yeh,
                  Konstantinos Psychas <kp2547@columbia.edu>, Teng Yang, Cong Han <ch3212@columbia.edu>

Copyright 2012-2020 Tingkai Liu, Mehmet Kerem, Aurel A. Lazar and Chung-Heng Yeh, Cong Han

## Problem 1: PRC of Connor-Stevens Neuron Model (Winfree's Method)
In this problem, you are asked to do the following:

1. Implement Winfree's method for approximating the Phase Response Curve (PRC) for Voltage of the CSN Model. Use simple words to describe the procedures and your method. 
2. Generate spike train with Connor-Stevens neuron and the I/O equivalent project-integrate-and-fire (PIF) neuron when the input is [$I_{ext}(t)$](#note-on-I-ext). Use simple words to describe your implementation of PIF. 
    - **Note**: You should compare the inter-spike-interval of the CSN model and the PIF model. 
3. Evaluate the mean $\mu$ and variance $\sigma^{2}$ of the difference between the corresponding spike times of the Connor-Stevens neuron and the equivalent PIF neuron for bandlimited stimulus described by [equation](#input) scaled to have maximum amplitude $c > 0$ (i.e. $\max_{t}|u(t)| = c$). Plot these statistics as a function of $c$ across values of $c$. 
    - **Note 1**: You should generate new inputs $u(t)$ (with different random coefficients) to ensure that the error statistics is not biased by the form of the input stimulus. 
    - **Note 2**: The value of $c$ should not be so large that the limit cycle of the ConnorStevens model collapses into singularity. It would be instructive to look at F-I curve of the ConnorStevens model first to find a range of input current values that gives periodic spiking.

### Note on $I_{ext} = I + u(t)$ <a name="note-on-I-ext"/>
Starting this homework, the injected current to model neurons $I_{ext}$ will have this form $I+u(t)$, where $I$ refers to _injected current_ (or _bias_ current) and $u(t)$ is the _stimulus_. 
The ideas is as follows:

1. $I$: injected (bias) current is a _constant_ value for all time that is injected to the neuron model. You can consider this value as the _average_/DC value of the input $I_{ext}$. Since this value is not time-varying, it contains _no information_ (in the sense of entropy). For this reason, we don't use refer to $I$ as _stimulus_. However, the bias $I$ will change the shape of the limit cycle $\Gamma$ (or the periodic solution ${\bf x}^0$) of the neuron model, where the higher the bias current $I$, the smaller the limit cycle and the faster the neuron oscillates. For this reaso, we can consider the dynamic of the neuron model to be _parametrized_/_indexed_ by the bias $I$.

2. $u(t)$: this is a time-varying signal that has information content. If we consider $I$ as the DC value of $I_{ext}$, then $u(t)$ will be a zero-mean signal that corresponds to local perturbation of the neuron dynamic around the limit cycle $\Gamma$. This is the input signal that we seeks to decode from the neuron spike times.

### Input Stimulus $u(t)$ - Complex Exponential <a name="input"></a>
The input stimulus here is a zero-DC valued complex exponential of order $M=5$ on support $t\in[-50, 200]$ ms with bandwidth $\Omega = 2\pi \cdot 20 \quad [rad\cdot s^-1]$,
$$
u(t)= \sum^{M}_{m=-M}a_{m} \exp\left(j\frac{m\Omega t}{M}\right)
$$

Use a uniform random number generate to obtain the complex coefficients, e.g. `rand` for the $a_{m},m=-M,\ldots,M$.
Note that for stimulus $u(t)$ to be real-valued signal, the coefficients $a_m$ need to be conjugate-symmetric. In another word, $a_m = \overline{a_{-m}} = Re(a_{-m}) - j Im(a_{-m})$, 
where $Re(\cdot), Im(\cdot)$ calculates the _real_ and _imagninary_ parts of the complex number respectively.

The stimulus $u(t)$ should be zero-DC valued, which means that $a_0 = 0$. An example of generating these coefficients is given below:
```MATLAB
am_right = rand(5,1) + 1i * rand(5,1);
a_m = [conj(flip(am_right)); 0; am_right];
```

This input stimulus $u(t)$ is additively coupled into a periodically spiking Connor-Stevens neuron (kernel code is provided).

## Problem 2: Synaptic Input and reduced PIF
So far, the injected currents into point neuron models are chosen to be arbitrary continuous waveforms. For this problem, you will explore using synaptic current to drive postsynaptic neurons.

You are asked to do the following:
1. Generate a population of `20` presynptic spike times $\{t^{i}_{k}\}$ using a Poisson process with constant rate $\lambda=50 Hz$. $k$ is index for spike-time, $i$ is index for pre-synaptic neuron.
2. Assume that 20 presynaptic spike train is each connected to a different postsynaptic ConnorStevens Model via an AMPA (first 10 spike trains) or NMDA (last 10 spike trains) Synapse, simulate this population of parallel synapse-neuron cascades. Generate the following plots
    1. Synaptic current for each postsynaptic neuron $I_{syn}^{i} = g^{i}(V^{i} - E^i_{rev})$
    2. Membrane Voltage for each postsynaptic neuron $V^{i}$
    3. Do you see any spiking behavior in the postsynaptic neurons? Why or why not?
3. Assume that all presynaptic spike trains are connected to the _same_ postsynaptic ConnorStevens Model through all 20 previously defined synapses, with the same parameters. Simulate this convergent synapse-neuron circuit where the synaptic current is the sum $I_{syn} = \sum_{i}g^{i}(V-E^i_{rev}), i=\{1,2,\ldots,20\}$.
4. Assuming that the model neuron is biased at a current level $I = mean(I_{syn})$ ($I_{syn}$ from step 3), implement reduced PIF neuron under this bias level $I$. Drive the reduced PIF neuron with input current $u(t) = I_{syn}(t)-I$. Plot the spike times of the model neuron and the reduced PIF. Is the reduced PIF a good approximation in this case? Why or why not?

### Parameters to be used in the problem
1. AMPA Synapse: $\alpha=1.1, \beta=0.19, \overline{g}_{max}=0.05, E_{rev}=0$
2. NMDA Synapse: $\alpha=0.072, \beta=0.0066, \overline{g}_{max}=0.01, E_{rev}=0$

The provided code specifies all parameters except for $\overline{g}_{max}$ and $E_{rev}$. 

### Synaptic Current $I_{syn}(t)= g_{syn}(t)\cdot(V_{post}(t)-E_{rev})$

Currents are injected into neurons through opening of ion-channels that may be controlled by membrane voltage of ligand bindings. In the case of chemical synapses (which is the case here), the ion-channels on post-synaptic neurons are open when neuro-transmitters relased by the pre-synaptic neurons bind to the receptors. There are many different kinds of such ligand-controlled ion-channels and they all have different dynamics, you are encouraged to look up concepts like Ionotropic vs Metabotropic. 

We can model all synaptic current going through ligand-gated ion channels as follows:
$$I_{syn}(t)= g_{syn}(t)\cdot(V_{post}(t)-E_{rev})$$
where 
1. $g_{syn}(t)$ is a time varying function that controls the synaptic conductance 
2. $V_{post}(t)$ is the membrane voltage of the postsynaptic neuron
3. $E_{rev}$ is the reverse potential of the synapse

**Note**: By convention, synaptic current $I_{syn}$ has negative value because it denotes _inward_ current to the neuron. Therefore, to use simulation kernels (e.g. `hodgkin_huxley_rinzel`), the input current $I_{ext}$ will be $I_{ext} = -I_{syn}$.

### Poisson Spiking $\lambda$, use the provided `utils.PoissonSpikes`
In time bin $[t,t+\Delta t]$, the probability of spike is $\lambda \cdot \Delta t$. Therefore, the spike state ( a binary number indicating if a in this time bin is $s(t) = \mathbb{1}_{x<\lambda \cdot \Delta t}$, where $x \sim Unif([0,1])$

## Setup

In [ ]:
addpath(genpath('./code'));

In [ ]:
%plot -b inline -f svg -r 96 -s 1080,720

In [ ]:
% Clear MATLAB Workspace first.
clearvars; close all; clc;

In [ ]:
% set RNG Seed
rng(20204020);

In [ ]:
set(groot,'defaulttextinterpreter','latex');
set(groot, 'defaultAxesTickLabelInterpreter','latex');
set(groot, 'defaultLegendInterpreter','latex');
default_fontsize = 13;
default_linewidth = 1.5;

# Problem 1 - PRC of Connor-Stevens Neuron Model (Winfree's Method)

## Initialization

In [ ]:
% TODO: Implement signal generator that takes in [time, coefficients, order, Bandwidth] as input
bandlmt_sig = @(t,a,M,omega) NaN

% NOTE: this spike_detect function takes in the threshold as input as well
spike_detect = @(v,threshold) [false, (v(2:end-1)>v(1:end-2) & v(2:end-1)>v(3:end) & v(2:end-1)>threshold), false];

In [ ]:
% TODO: Specify the time resolution for computing PRC. 
% The smaller the value the better the Winfree method performs

% NOTE: start with a smaller value to ensure the code works before making it larger
dt = NaN;

In [ ]:
% TODO: Specify the bias of Connor-Steven's Model (I in I_ext = I + u(t)).
% This value will be used in both the neuron model itself and its PIF.
bias = NaN;

## Item (i): Extract the PRC

In [ ]:
%% TODO: Implement Winfree's Method
% As a suggestion, the function should return 
% 1. period: the period of oscillation, maybe in number of time-steps
% 2. limitCycle: the output of the state variables [V, n, m, h, a, b] on a limit cycle
% 3. PRC: the phase response curve of the Voltage along the limit cycle
[period, limitCycle, PRC] = winfree(NaN)


In [ ]:
%% TODO: Visualize the limit cycle and the PRC along it

## Item (ii): Compare Spikes of PIF and CSN

### Output of CSN
We first generate a bandlimited signal with random coefficients

In [ ]:
% temporal support of the signal [-50, 200] ms
t = -50/1e3:dt:200/1e3;

% bandwidth: 20Hz
Omega = 2*pi*20;

% order: 5
M = 5;

% Amplitude of the signal, you can use other values.
Sig_Amp = 1;

% generate coefficient
am = NaN

% generate input signal. Make sure that the signal's DC value (mean) is 0
u1 = bandlmt_sig(t, am, M, Omega);
u1 = Sig_Amp*u1/max(u1);

Simulate the CSN with bias and bandlimited input added.

In [ ]:
% TODO: simulate model and find spikes
[V1, W] = connor_stevens(NaN);
tk_index1 = NaN;

### Output of PIF
The t-transform of the PIF neuron can be written as,
$$
\int_{t_k}^{t_{k+1}}(1+\psi(s+\tau(s))u(s))ds = \delta_{t_{k+1}}-\delta_{t_{k}} \approx T, \\
\tau(t) = \psi(t+\tau(t))u(t),~\tau(0)=0.
$$    
We assume that $\tau(s)=0$ , and hence the $t$-transform is reduced to
$$ 
\int_{t_k}^{t_{k+1}}(1+\psi(s)u(s))ds = T.
$$  

In [ ]:
% TODO: Implement and execute the PIF model using the input defined above and the PRC curve computed

In [ ]:
% TODO: Plot CSN, PIF outputs and compare the Inter-spike intervals

## Item (iii): Record PIF Error Statistics

In [ ]:
% TODO: Initalize experiment
% Find a range of amplitude $C$ that is within the range of permissable input current to the CSN model.
% Too large and the limit cycle could collapse
C = NaN; % amplitude of u(t)
avg_diff = zeros(size(C));
var_diff = zeros(size(C));

In [ ]:
% TODO: Simulate the CSN and PIF for inputs I_ext = I + u(t) for different amplitude of u(t): max|u(t)| = C
% TODO: Compute the difference between interspike interval of the CSN and PIF, and calculate the mean/standard deviation of the error.
% TODO: visualize the error across stimulus amplitude

## Problem 2: Synaptic Input and reduced PIF

### Step 1. Generate & Plot Spike Trains

In [ ]:
% TODO: generate 20 spike trains
spike_state = NaN;

In [ ]:
% TODO: Plot Spike train

### Step 2. Simulate Synapses

In [ ]:
% TODO: Convert spike trains to NeuroTransmitter concentration [NT](t)

NT = NaN;

In [ ]:
params_ampa = struct('gmax',0.05,'E',0);
params_nmda = struct('gmax',0.01,'E',0);

#### Step 2.1 Simulate AMPA first


In [ ]:
for n = 1:NChannels/2
    V = zeros(size(t));
    x = zeros(5, numel(t));
    
    % initial condition for ConnorStevens
    V(1) = -67.97;
    x(:,1) = [0.1559; 0.01008; 0.9659; 0.5404; 0.2885];
    for tt = 2:length(t)
        % TODO:
        % 1. simulate state and conductance of AMPA synapse
        % 2. generate I_syn for a given channel
        % 3. update connorsteven's state and voltage for that channel
    end
    % TODO 4. find spike states of the postsynaptic neuron
end

#### Step 2.2 Simulate NMDA next

In [ ]:
for n = NChannels/2+1:NChannels
    V = zeros(size(t));
    x = zeros(5, numel(t));
    
    % initial condition for ConnorStevens
    V(1) = -67.97;
    x(:,1) = [0.1559; 0.01008; 0.9659; 0.5404; 0.2885];
    for tt = 2:length(t)
        % TODO:
        % 1. simulate state and conductance of NMDA synapse
        % 2. generate I_syn for a given channel
        % 3. update connorsteven's state and voltage for that channel
    end
    % TODO 4. find spike states of the postsynaptic neuron

end

#### Step 2.3 Plot Outputs

In [ ]:
% TODO: Plot

In [ ]:
% TODO: Does it spike? Why or why not?

### Step 3. Run Convergent Circuit 
All synapses provided input to the same output neuron

In [ ]:
%TODO: base your code on the code from Step 2.

In [ ]:
%TODO: Plot

### Step 4. Reudced PIF 

In [ ]:
% TODO: Use synaptic current from the previous step to find the bias current and signal.
% TODO: Find the period and PRC of the connorstevens model at the given bias current

In [ ]:
% TODO: Simulate PIF using the PRC found above and the signal deteremined from the synaptic current in Step 3

In [ ]:
% TODO: Plot

In [ ]:
% TODO: Is PIF a good approximation? Why and why not? 
% HINT: think about the difference in the assumption about the input 
%     between the standard PIF and the synaptic input used here.